# Test 7 (Simulated Annealing, with loops)

Field: 8mT along the x axis

In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import random
from math import isclose

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # Ignore some messages from pandas

from tqdm import tqdm
import matplotlib.pyplot as plt

sys.path.insert(0, '../icenumerics/')
import icenumerics as ice
from auxiliary import *
from montecarlo_tools import *

ureg = ice.ureg

%reload_ext autoreload
%autoreload 2

idx = pd.IndexSlice

In [ ]:
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'

In [ ]:
# Definition of colloids

sp = ice.spins()

trapSep = 10*ureg.um
particleRadius = 5*ureg.um
L = 30*ureg.um
N = 10

# Create simulation objects
sp.create_lattice("square",[N,N],lattice_constant=L, border="periodic")

particle = ice.particle(radius = particleRadius,
            susceptibility = 0.0576,
            diffusion = 0.125*ureg.um**2/ureg.s,
            temperature = 300*ureg.K,
            density = 1000*ureg.kg/ureg.m**3)

trap = ice.trap(trap_sep = trapSep,
            height = 4*ureg.pN*ureg.nm,
            stiffness = 1e-3*ureg.pN/ureg.nm)

col = ice.colloidal_ice(sp, particle, trap,
            height_spread = 0, 
            susceptibility_spread = 0.1,
            periodic = True)


col.randomize()
col.region = np.array([[0,0,-3*(particleRadius/L/N).magnitude],[1,1,3*(particleRadius/L/N).magnitude]])*N*L

# Simulated Annealing

Try to optimize this algorithm

* See which functions are slow
* See which deep copies can be omitted.

In [ ]:
def simulation(colloid, niter = 100):

    B0 = 8*ureg.mT
    field = [1,0,0]*B0

    dataPath = "../data/test7/"

    # Initialize stuff
    energies = np.zeros(niter)

    # Save initial state
    colloid.to_ctrj().to_csv(dataPath + "trj0.csv")

    # No sé por qué debo de calcular esto primero xd
    Eold = colloid.calculate_energy(B = field).magnitude
    energies[0] = Eold

    T = 300

    for k in tqdm(range(1,niter)):
        
        newcolloid = colloid.copy(deep = True)
        newcolloid = flip_colloids(newcolloid,amount=3)
        newcolloid = flipLoop(newcolloid,a=30,size=10)

        Enew = newcolloid.calculate_energy(B=field).magnitude
        dE = Enew - Eold

        # Accept or reject the change
        if isaccepted(dE,T):
            colloid = newcolloid.copy(deep = True)
            #colloid = newcolloid
            Eold = Enew
            energies[k] = Eold
        else:
            energies[k] = Eold
        
        
        currentFile = f"trj{k}.csv"
        colloid.to_ctrj().to_csv(dataPath + currentFile)
        
        T = 0.9*T
    
    return colloid, energies


In [ ]:
np.random.seed(0)
final_state, data = simulation(col,niter=10)

## Saving data

In [ ]:
dataPath = "../data/test7/"
pd.DataFrame(data).to_csv(dataPath + "data.csv")

## Plotting energy

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
ax.plot(data)
ax.set_ylabel("Energy")
ax.set_xlabel("MC Steps")

In [ ]:
data[-1]

In [ ]:
B0 = 8*ureg.mT
field = [1,0,0]*B0
final_state.calculate_energy(B=field)

## Plotting states

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
v = ice.vertices()
v = v.colloids_to_vertices(col)
col.display(ax)
v.display()
fig.savefig("initial.png")

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
v = ice.vertices()
v = v.colloids_to_vertices(final_state)
final_state.display(ax)
v.display()
fig.savefig("final.png")

# Loading experiments and performing some analysis

In [ ]:
final_trj = pd.read_csv("../data/test6/r2/trj9999.csv", index_col=0)
energies = pd.read_csv("../data/test6/r2/energies.csv", index_col=0)
final_trj.head()

In [ ]:
col = get_colloids_from_ctrj(final_trj,particle,trap)
col.region = np.array([[0,0,-3*(particleRadius/L/N).magnitude],[1,1,3*(particleRadius/L/N).magnitude]])*N*L

v = ice.vertices()
v = v.colloids_to_vertices(col)
v.vertices.head()

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
col.display(ax)
v.display(ax)

In [ ]:
energy = getEnergy(col)
print(energy)
energies.tail()

In [ ]:
vertices = classifyVertices(v.vertices)
count_vertices_single(vertices)

# Suffering flippling colloids

In [ ]:
col2 = col.copy(deep=True) 

In [ ]:
indices = [90,99,191,193,195,197,199,186,162,61,62,151,152,153,159,50,51,52,57,58,59,141,142,147,148,149,
           40,41,42,46,47,48,49,131,132,133,136,137,138,139,
           30,31,32,33,35,36,37,38,39,121,122,123,126,127,128,129,
           20,21,22,23,24,25,26,27,28,29,111,112,113,114,115,116,117,118,119,
           10,11,12,13,14,15,16,17,18,19,101,102,103,104,105,106,107,108,109,
           180,150,130,120,110,100,
           0,1,2,3,4,5,6,7,8,9
           ]



col2 = flipColloids_norandom(col2,indices)


In [ ]:
v = ice.vertices()
v = v.colloids_to_vertices(col2)

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
col2.display(ax)
v.display(ax)

In [ ]:
gs = col2.to_ctrj()

In [ ]:
gs.to_csv("../data/gs90deg.csv")

In [ ]:
E0 = getEnergy(col)
E1 = getEnergy(col2)
E1 - E0

# Test stuff

Random experimental stuff that is not relevant for this test xd.

* Put all mcsteps in a multi index frame

Here I was trying the idea above but failed haha

In [ ]:
state1 = col.to_ctrj()
state2 = flipColloids(col,10).to_ctrj()

mcsteps = [k for k in range(2)]
particles = state1.index.get_level_values("id").values

index = pd.MultiIndex.from_product([mcsteps,particles], names=["mcstep","id"])
trj = pd.DataFrame(columns=state1.columns.to_list(), index=index)
trj